<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Let us consider the problem of minimizing the Rosenbrock function. This
function (and its respective derivatives) is implemented in `rosen`
(resp. `rosen_der`, `rosen_hess`) in the `scipy.optimize`.


In [ ]:
from scipy.optimize import minimize, rosen, rosen_der

A simple application of the *Nelder-Mead* method is:


In [ ]:
x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
res = minimize(rosen, x0, method='Nelder-Mead', tol=1e-6)
res.x

array([ 1.,  1.,  1.,  1.,  1.])

Now using the *BFGS* algorithm, using the first derivative and a few
options:


In [ ]:
res = minimize(rosen, x0, method='BFGS', jac=rosen_der,
               options={'gtol': 1e-6, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 26
         Function evaluations: 31
         Gradient evaluations: 31

In [ ]:
res.x

array([ 1.,  1.,  1.,  1.,  1.])

In [ ]:
print(res.message)

Optimization terminated successfully.

In [ ]:
res.hess_inv

array([
    [ 0.00749589,  0.01255155,  0.02396251,  0.04750988,  0.09495377],  # may vary
    [ 0.01255155,  0.02510441,  0.04794055,  0.09502834,  0.18996269],
    [ 0.02396251,  0.04794055,  0.09631614,  0.19092151,  0.38165151],
    [ 0.04750988,  0.09502834,  0.19092151,  0.38341252,  0.7664427 ],
    [ 0.09495377,  0.18996269,  0.38165151,  0.7664427,   1.53713523]
])

Next, consider a minimization problem with several constraints (namely
Example 16.4 from [5]). The objective function is:


In [ ]:
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2

There are three constraints defined as:


In [ ]:
cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2 * x[1] + 2},
        {'type': 'ineq', 'fun': lambda x: -x[0] - 2 * x[1] + 6},
        {'type': 'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2})

And variables must be positive, hence the following bounds:


In [ ]:
bnds = ((0, None), (0, None))

The optimization problem is solved using the SLSQP method as:


In [ ]:
res = minimize(fun, (2, 0), method='SLSQP', bounds=bnds, constraints=cons)

It should converge to the theoretical solution ``[1.4 ,1.7]``. *SLSQP* also
returns the multipliers that are used in the solution of the problem. These
multipliers, when the problem constraints are linear, can be thought of as the
Karush-Kuhn-Tucker (KKT) multipliers, which are a generalization
of Lagrange multipliers to inequality-constrained optimization problems ([20]).

Notice that at the solution, the first constraint is active. Let's evaluate the
function at solution:


In [ ]:
cons[0]['fun'](res.x)

np.float64(1.4901224698604665e-09)

Also, notice that at optimality there is a non-zero multiplier:


In [ ]:
res.multipliers

array([0.8, 0. , 0. ])

This can be understood as the local sensitivity of the optimal value of the
objective function with respect to changes in the first constraint. If we
tighten the constraint by a small amount ``eps``:


In [ ]:
eps = 0.01
cons[0]['fun'] = lambda x: x[0] - 2 * x[1] + 2 - eps

we expect the optimal value of the objective function to increase by
approximately ``eps * res.multipliers[0]``:


In [ ]:
eps * res.multipliers[0]  # Expected change in f0

np.float64(0.008000000027153205)

In [ ]:
f0 = res.fun  # Keep track of the previous optimal value
res = minimize(fun, (2, 0), method='SLSQP', bounds=bnds, constraints=cons)
f1 = res.fun  # New optimal value
f1 - f0

np.float64(0.008019998807885509)